# Hyperlink graph related experiments 

Creating Hyperlink graph from mongo crawled data.
Importing graph as required in NetworkX.
Running Link analysis algo such as (pagerank, hits).
Using Gephi for visualization and further analysis like (community detection).

In [1]:
# importing all the modules
from pymongo import MongoClient
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import write_dot
from pymongo import MongoClient
from simhash import Simhash
from urlparse import urlparse
import redis
import operator
import pandas as pd

In [2]:
# mongo connnection 
client = MongoClient('10.240.0.68',36018)
collection_name = client['ayush']['abbvie_hospital_affiliation_meta_may12']

# redis connection
cache = redis.StrictRedis(host='localhost',port=6379, db=0)
cache2=redis.StrictRedis(host='localhost',port=6379, db=0)

In [3]:
# creaing toy graph from NetworkX
G = nx.Graph()
G.add_node(1)
G.add_nodes_from([2,3])
H = nx.path_graph(10)
G.add_nodes_from(H)
G.add_node(H)
G.add_edge(1,2)
e = (2,3)
G.add_edge(*e)
G.add_edges_from([(1,2),(1,3)])

In [4]:
# different types of graph and plotting using matplotlib

# G.neighbors(1)
petersen=nx.petersen_graph()
tutte=nx.tutte_graph()
maze=nx.sedgewick_maze_graph()
tet=nx.tetrahedral_graph()
K_5=nx.complete_graph(5)
K_3_5=nx.complete_bipartite_graph(3,5)
barbell=nx.barbell_graph(10,10)
lollipop=nx.lollipop_graph(10,20)
er=nx.erdos_renyi_graph(100,0.15)
ws=nx.watts_strogatz_graph(30,3,0.1)
nx.draw(barbell)
plt.show()

/usr/lib/pymodules/python2.7/matplotlib/collections.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors_original != 'face':
/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [7]:
# reading and writing graphs from edgelist method
nx.write_edgelist(nx.path_graph(4), "test.edgelist")
G=nx.read_edgelist("hyperlink_url_parent.edgelist",create_using=nx.DiGraph())
nx.draw(G)
plt.show()

MemoryError: 

In [7]:
# number of nodes,edges
print link_graph.number_of_nodes()
print link_graph.number_of_edges()

# Pagerank
pr = nx.pagerank(link_graph, alpha=0.85)

# HITS
# h,a = nx.hits(link_graph)

NameError: name 'link_graph' is not defined

In [19]:
# sort a dictionary

# sorted_x = sorted(h.items(), key=operator.itemgetter(1))
url_score = []
for val in sorted(pr.items(), key=lambda x: (-x[1], x[0])):
    if val[1] > 0.0:
        url =  url_hash_lookup(val[0])
        url_score.append([url,val[1]])
df = pd.DataFrame(url_score, columns=["url","pr_rank"])
df.to_csv('page_rank.csv', index=False)

In [12]:
# scores writing to csv
pages = []
pages.append(['abc',123])
df = pd.DataFrame(pages, columns=["url","pr_score"])
df.to_csv('sample.csv', index=False)

In [18]:
# method to read documents from mongo collection and create an Index file of urls/referer_urls 
def read_referer_url(coll_name):
    edge_list_filename = open('hyperlink_graph_abbvie_edgelist.edgelist','w')
    document_cursor = coll_name.find({},{'html':0,'text':0})
    
    for document in document_cursor:
        edge_line = ''
        if 'referer_url' in document:
            referer_url = document['referer_url']
            scheme_stripped_referer_url = referer_url.split('://')[1]
            
            if '/' in scheme_stripped_referer_url[-1]:
                scheme_stripped_referer_url = scheme_stripped_url[0:-1]
            
            # get simhash of the url
            hash_url = generate_simhash(scheme_stripped_referer_url)
            edge_line+=hash_url+' '
            store_url_hash(hash_url,scheme_stripped_referer_url)
            
        if  'url' in document:
            url = document['url']
            scheme_stripped_url = url.split('://')[1]
            
            if '/' in scheme_stripped_url[-1]:
                scheme_stripped_url = scheme_stripped_url[0:-1]
            
            # get simhash of the url
            hash_url = generate_simhash(scheme_stripped_url)
            edge_line+=hash_url
            store_url_hash(hash_url,scheme_stripped_url)
        
        if (edge_line and edge_line!=''):
            edge_list_filename.write(edge_line+'\n')
            
read_referer_url(collection_name)

In [12]:
# create simhash of the tags
def generate_simhash(data):
        simhash_string = str(Simhash(data).value)
#         simhash_binary = bin(Simhash(data).value)
#         return simhash_string, simhash_binary
        return simhash_string
print generate_simhash("".join("www.nice.org.uks/"))

11971568802140945701


In [19]:
# create a directed graph in NetworkX using EdgeList DS file
def create_directed_graph(edge_list_filename):
    url_graph = nx.read_edgelist(edge_list_filename,create_using=nx.DiGraph())
    return url_graph
link_graph = create_directed_graph('hyperlink_graph_abbvie_edgelist.edgelist')

In [23]:
# function to write a graph in GML for persistent storage and usable some other tools in future
def store_graph_gml(graph):
    nx.write_gml(graph, "hyperlink_graph_abbvie.gml")
    
store_graph_gml(link_graph)

In [3]:
# function to read GML grpah
def read_graph_gml(gml_graph_filename):
    gml_graph = nx.read_gml(gml_graph_filename)
    return gml_graph

link_graph = read_graph_gml('hyperlink_graph_abbvie.gml')

In [16]:
# store the url and hash values for future lookup (in redis)
def store_url_hash(hash_value,url):
    cache.setnx(hash_value,url)
    cache2.setnx()

In [9]:
cache.setnx('foo','bar')
cache.setnx('foo1','bar2')
cache.get('foo1')

'bar2'

In [6]:
def url_hash_lookup(hash_value):
    return cache.get(hash_value)

url_hash_lookup('290356238619843879')

In [19]:
# reading deny domains file
deny_domains_list = []
with open('../FocusedCrawling/deny_domains','r') as fp:
    deny_domains_list = fp.readlines()
    deny_domains_list = [d.strip() for d in deny_domains_list]

print deny_domains_list

['youtube.com', 'google.com', 'twitter.com', 'tumblr.com', 'facebook.com', 'linkedin.com', 'pinterest.com', 'instagram.com', 'flickr.com', 'reddit.com', 'youtube.de', 'google.de', 'twitter.de', 'tumblr.de', 'facebook.de', 'linkedin.de', 'pinterest.de', 'instagram.de', 'flickr.de', 'reddit.de']
